# step_renderer

> Phase 1 step renderer: Source Selection & Ordering with two-column layout and collapsible preview

In [ ]:
#| default_exp components.step_renderer

In [ ]:
#| export
from typing import Any, Dict, List, Optional

from fasthtml.common import (
    Div, H2, P, Span, Input, Script, Details, Summary
)

# DaisyUI components
from cjm_fasthtml_daisyui.components.data_display.badge import badge, badge_colors, badge_sizes
from cjm_fasthtml_daisyui.components.data_display.collapse import (
    collapse, collapse_title, collapse_content, collapse_modifiers
)
from cjm_fasthtml_daisyui.components.navigation.tabs import tabs, tabs_styles, tab
from cjm_fasthtml_daisyui.utilities.semantic_colors import bg_dui, text_dui, border_dui, ring_dui
from cjm_fasthtml_daisyui.utilities.border_radius import border_radius

# Tailwind utilities
from cjm_fasthtml_tailwind.utilities.spacing import p, m
from cjm_fasthtml_tailwind.utilities.sizing import w, h, min_h, min_w
from cjm_fasthtml_tailwind.utilities.typography import font_size, font_weight
from cjm_fasthtml_tailwind.utilities.layout import overflow
from cjm_fasthtml_tailwind.utilities.borders import border
from cjm_fasthtml_tailwind.utilities.effects import ring
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import (
    flex_display, flex_direction, justify, items, gap, grow
)
from cjm_fasthtml_tailwind.core.base import combine_classes

# Local imports
from cjm_transcript_source_select.html_ids import SelectionHtmlIds
from cjm_transcript_source_select.models import SelectionUrls
from cjm_transcript_source_select.utils import count_words

# Keyboard navigation library
from cjm_fasthtml_keyboard_navigation.core.focus_zone import FocusZone
from cjm_fasthtml_keyboard_navigation.core.actions import KeyAction
from cjm_fasthtml_keyboard_navigation.core.manager import ZoneManager
from cjm_fasthtml_keyboard_navigation.core.navigation import LinearVertical
from cjm_fasthtml_keyboard_navigation.components.system import render_keyboard_system
from cjm_fasthtml_keyboard_navigation.components.hints import render_keyboard_hints

# Selection component imports
from cjm_transcript_source_select.components.helpers import (
    _generate_sortable_init_script
)
from cjm_transcript_source_select.components.source_browser import (
    _render_source_browser
)
from cjm_transcript_source_select.components.selection_queue import (
    _render_selection_queue
)
from cjm_transcript_source_select.components.preview_panel import (
    _render_preview_panel
)
from cjm_transcript_source_select.components.local_files import (
    _render_local_files_browser, _create_db_browser_config
)

# File browser library
from cjm_fasthtml_file_browser.providers.local import LocalFileSystemProvider
from cjm_fasthtml_file_browser.core.models import BrowserState

## Keyboard Configuration

Factory functions to create keyboard navigation zones, actions, and manager for Phase 1.

In [ ]:
#| export
# Hidden input IDs for keyboard navigation (must match route handler parameter names)
SD_FOCUSED_RECORD_ID_INPUT = "sd-focused-record-id"
SD_FOCUSED_PROVIDER_ID_INPUT = "sd-focused-provider-id"

# Button IDs for keyboard actions
SD_TOGGLE_BTN = "sd-toggle-btn"
SD_REMOVE_BTN = "sd-remove-btn"
SD_REORDER_UP_BTN = "sd-reorder-up-btn"
SD_REORDER_DOWN_BTN = "sd-reorder-down-btn"
SD_TAB_PREV_BTN = "sd-tab-prev-btn"
SD_TAB_NEXT_BTN = "sd-tab-next-btn"
SD_PREVIEW_BTN = "sd-preview-btn"


def _create_selection_keyboard_manager() -> ZoneManager:  # Configured keyboard zone manager
    """Create the keyboard zone manager for Phase 1 selection step."""
    # Source browser zone (left panel)
    browser_zone = FocusZone(
        id=SelectionHtmlIds.SOURCE_LIST,
        item_selector='tr[data-selectable="true"]',
        navigation=LinearVertical(),
        data_attributes=("record-id", "provider-id"),
        zone_focus_classes=(str(ring(1)), str(ring_dui.primary)),
        item_focus_classes=(str(bg_dui.primary.opacity(10)), str(ring(1)), str(ring_dui.primary)),
        on_focus_change="triggerPreviewUpdate",
        hidden_input_prefix="sd-focused",
    )
    
    # Selection queue zone (right panel)
    queue_zone = FocusZone(
        id=SelectionHtmlIds.QUEUE_CONTAINER,
        item_selector="li.queue-item",
        navigation=LinearVertical(),
        data_attributes=("record-id", "provider-id"),
        zone_focus_classes=(str(ring(1)), str(ring_dui.secondary)),
        item_focus_classes=(str(bg_dui.secondary.opacity(10)), str(ring(1)), str(ring_dui.secondary)),
        on_focus_change="triggerPreviewUpdate",
        hidden_input_prefix="sd-focused",
    )
    
    # Define keyboard actions
    actions = (
        # Toggle selection (Space/Enter) - browser panel only
        KeyAction(
            key=" ",
            htmx_trigger=SD_TOGGLE_BTN,
            zone_ids=(SelectionHtmlIds.SOURCE_LIST,),
            description="Toggle selection",
            hint_group="Selection",
        ),
        KeyAction(
            key="Enter",
            htmx_trigger=SD_TOGGLE_BTN,
            zone_ids=(SelectionHtmlIds.SOURCE_LIST,),
            description="Toggle selection",
            hint_group="Selection",
            show_in_hints=False,
        ),
        
        # Remove from queue (Delete/Backspace) - queue panel only
        KeyAction(
            key="Delete",
            htmx_trigger=SD_REMOVE_BTN,
            zone_ids=(SelectionHtmlIds.QUEUE_CONTAINER,),
            description="Remove from queue",
            hint_group="Queue",
        ),
        KeyAction(
            key="Backspace",
            htmx_trigger=SD_REMOVE_BTN,
            zone_ids=(SelectionHtmlIds.QUEUE_CONTAINER,),
            description="Remove from queue",
            hint_group="Queue",
            show_in_hints=False,
        ),
        
        # Reorder queue (Shift+Arrow) - queue panel only
        KeyAction(
            key="ArrowUp",
            modifiers=frozenset({"shift"}),
            htmx_trigger=SD_REORDER_UP_BTN,
            zone_ids=(SelectionHtmlIds.QUEUE_CONTAINER,),
            description="Move up in queue",
            hint_group="Queue",
        ),
        KeyAction(
            key="ArrowDown",
            modifiers=frozenset({"shift"}),
            htmx_trigger=SD_REORDER_DOWN_BTN,
            zone_ids=(SelectionHtmlIds.QUEUE_CONTAINER,),
            description="Move down in queue",
            hint_group="Queue",
        ),
        
        # Tab switching (Ctrl+Shift+[ and Ctrl+Shift+]) - browser panel only
        # Note: When Shift is held, [ becomes { and ] becomes } in event.key
        KeyAction(
            key="{",
            modifiers=frozenset({"ctrl", "shift"}),
            htmx_trigger=SD_TAB_PREV_BTN,
            zone_ids=(SelectionHtmlIds.SOURCE_LIST,),
            description="Previous tab",
            hint_group="Tabs",
        ),
        KeyAction(
            key="}",
            modifiers=frozenset({"ctrl", "shift"}),
            htmx_trigger=SD_TAB_NEXT_BTN,
            zone_ids=(SelectionHtmlIds.SOURCE_LIST,),
            description="Next tab",
            hint_group="Tabs",
        ),
    )
    
    return ZoneManager(
        zones=(browser_zone, queue_zone),
        actions=actions,
        prev_zone_key="ArrowLeft",
        next_zone_key="ArrowRight",
        on_zone_change="onZoneChange",
        state_hidden_inputs=True,  # Enable JavaScript to update state hidden inputs
    )

## Keyboard Hints Component

In [ ]:
#| export
def _render_selection_keyboard_hints(
    manager: ZoneManager,  # Keyboard zone manager with actions configured
) -> Any:  # Collapsible keyboard hints component
    """Render keyboard shortcut hints in a collapsible container."""
    hints = render_keyboard_hints(
        manager,
        include_navigation=True,
        include_zone_switch=True,
        badge_style="outline",
        container_id="sd-keyboard-hints",
        use_icons=False
    )
    
    return Details(
        Summary(
            "Keyboard Shortcuts",
            cls=combine_classes(collapse_title, font_size.sm, font_weight.medium)
        ),
        Div(
            hints,
            cls=collapse_content
        ),
        cls=combine_classes(collapse, collapse_modifiers.arrow, bg_dui.base_200)
    )

## Footer Component

In [ ]:
#| export
def _render_selection_stats(
    selected_sources: List[Dict[str, str]],  # Selected sources
    transcriptions: List[Dict[str, Any]],  # All transcriptions (for word count)
    oob: bool = False,  # Whether to render as OOB swap
) -> Any:  # Stats component
    """Render the selection statistics (word count and source count)."""
    # Calculate total word count
    total_words = 0
    selected_record_ids = {s.get("record_id") for s in selected_sources}
    for t in transcriptions:
        if t.get("record_id") in selected_record_ids:
            total_words += count_words(t.get("text", ""))
    
    source_count = len(selected_sources)
    
    return Div(
        Span(
            f"Total: {total_words:,} words from {source_count} source{'s' if source_count != 1 else ''}",
            cls=combine_classes(font_size.sm, text_dui.base_content.opacity(70))
        ) if source_count > 0 else Span(
            "Select sources to continue",
            cls=combine_classes(font_size.sm, text_dui.base_content.opacity(50))
        ),
        id=SelectionHtmlIds.SELECTION_STATS,
        hx_swap_oob="true" if oob else None,
    )

In [ ]:
#| export
def _render_selection_footer(
    selected_sources: List[Dict[str, str]],  # Selected sources
    transcriptions: List[Dict[str, Any]],  # All transcriptions (for word count)
) -> Any:  # Footer component
    """Render the footer with statistics and continue button."""
    return Div(
        # Statistics
        _render_selection_stats(selected_sources, transcriptions, oob=False),
        
        id=SelectionHtmlIds.SELECTION_FOOTER,
        cls=combine_classes(
            p(4),
            bg_dui.base_100,
            border_dui.base_300,
            border.t(),
            flex_display,
            justify.between,
            items.center
        )
    )

## Tab Components

In [ ]:
#| export
def _render_tab_headers(
    active_tab: str,  # Currently active tab ('db' or 'files')
    tab_switch_url: str = "",  # URL for switching tabs via HTMX
    oob: bool = False,  # Whether to render as OOB swap
) -> Any:  # Tab headers container
    """Render the tab header radio inputs."""
    return Div(
        Input(
            type="radio",
            name="source_tabs",
            aria_label="Plugin DB",
            checked="checked" if active_tab == "db" else None,
            cls=str(tab),
            id=SelectionHtmlIds.SOURCE_TAB_DB,
            tabindex="-1",
            onfocus="this.blur()",
            # Always include HTMX (server handles if already on this tab)
            hx_post=tab_switch_url if tab_switch_url else None,
            hx_vals='{"direction": "db"}' if tab_switch_url else None,
            hx_target=SelectionHtmlIds.as_selector(SelectionHtmlIds.SOURCE_TAB_CONTENT) if tab_switch_url else None,
            hx_swap="innerHTML" if tab_switch_url else None,
        ),
        Input(
            type="radio",
            name="source_tabs",
            aria_label="Local Files",
            checked="checked" if active_tab == "files" else None,
            cls=str(tab),
            id=SelectionHtmlIds.SOURCE_TAB_FILES,
            tabindex="-1",
            onfocus="this.blur()",
            # Always include HTMX (server handles if already on this tab)
            hx_post=tab_switch_url if tab_switch_url else None,
            hx_vals='{"direction": "files"}' if tab_switch_url else None,
            hx_target=SelectionHtmlIds.as_selector(SelectionHtmlIds.SOURCE_TAB_CONTENT) if tab_switch_url else None,
            hx_swap="innerHTML" if tab_switch_url else None,
        ),
        cls=combine_classes(tabs, tabs_styles.box),
        role="tablist",
        id=SelectionHtmlIds.SOURCE_TABS,
        hx_swap_oob="true" if oob else None,
    )

In [ ]:
#| export
def _render_source_tabs(
    active_tab: str,  # Currently active tab ('db' or 'files')
    active_content: Any,  # Content for the currently active tab
    tab_switch_url: str = "",  # URL for switching tabs via HTMX
) -> Any:  # Tabs header + separate content container
    """Render source type tabs with a single shared content container."""
    # Tab headers
    tab_headers = _render_tab_headers(active_tab, tab_switch_url, oob=False)
    
    # Content container (separate from tabs)
    content_container = Div(
        active_content,
        id=SelectionHtmlIds.SOURCE_TAB_CONTENT,
        cls=combine_classes(
            grow(),
            min_h(0),
            min_w._0,  # Allow content to shrink below intrinsic width
            bg_dui.base_100,
            border_radius.box
        )
    )
    
    # Return both as a fragment (wrapper div for layout)
    return Div(
        tab_headers,
        content_container,
        cls=combine_classes(
            grow(),
            min_h(0),
            min_w._0,  # Allow tabs container to shrink below intrinsic width
            flex_display,
            flex_direction.col
        )
    )

## Main Step Renderer

In [ ]:
#| export
from fasthtml.common import Button

from cjm_fasthtml_tailwind.utilities.layout import display_tw


# Module-level provider and config for step renderer
_step_renderer_provider: Optional[LocalFileSystemProvider] = None
_step_renderer_config = None

In [ ]:
#| export
def _get_step_renderer_provider() -> LocalFileSystemProvider:
    """Get or create the local files provider for step renderer."""
    global _step_renderer_provider
    if _step_renderer_provider is None:
        _step_renderer_provider = LocalFileSystemProvider()
    return _step_renderer_provider

In [ ]:
#| export
def _get_step_renderer_config():
    """Get or create the local files config for step renderer."""
    global _step_renderer_config
    if _step_renderer_config is None:
        _step_renderer_config = _create_db_browser_config()
    return _step_renderer_config

In [ ]:
#| export
def render_selection_step(
    sources: List[Dict[str, Any]],  # Available source plugins
    transcriptions: List[Dict[str, Any]],  # Available transcription records
    selected_sources: List[Dict[str, str]],  # Ordered selection
    grouping_mode: str,  # Grouping mode: "media_path" or "batch_id"
    external_db_paths: List[str],  # External database paths
    file_browser_state: Dict[str, Any],  # Serialized BrowserState from file-browser library
    active_tab: str,  # Active tab: "db" or "files"
    urls: SelectionUrls,  # URL bundle for selection routes
) -> Any:  # FastHTML component
    """Render Phase 1: Source Selection & Ordering step with two-column layout."""
    # Create keyboard manager
    kb_manager = _create_selection_keyboard_manager()
    
    # Build keyboard system with URL mappings and vals_map for direction parameters
    kb_system = render_keyboard_system(
        kb_manager,
        url_map={
            SD_TOGGLE_BTN: urls.toggle_focused,
            SD_REMOVE_BTN: urls.remove,
            SD_REORDER_UP_BTN: urls.keyboard_reorder or urls.reorder,
            SD_REORDER_DOWN_BTN: urls.keyboard_reorder or urls.reorder,
            SD_TAB_PREV_BTN: urls.tab_switch,
            SD_TAB_NEXT_BTN: urls.tab_switch,
        },
        target_map={
            SD_TOGGLE_BTN: SelectionHtmlIds.as_selector(SelectionHtmlIds.QUEUE_CONTAINER),
            SD_REMOVE_BTN: SelectionHtmlIds.as_selector(SelectionHtmlIds.QUEUE_CONTAINER),
            SD_REORDER_UP_BTN: SelectionHtmlIds.as_selector(SelectionHtmlIds.QUEUE_CONTAINER),
            SD_REORDER_DOWN_BTN: SelectionHtmlIds.as_selector(SelectionHtmlIds.QUEUE_CONTAINER),
            SD_TAB_PREV_BTN: SelectionHtmlIds.as_selector(SelectionHtmlIds.SOURCE_TAB_CONTENT),
            SD_TAB_NEXT_BTN: SelectionHtmlIds.as_selector(SelectionHtmlIds.SOURCE_TAB_CONTENT),
        },
        swap_map={
            SD_TAB_PREV_BTN: "innerHTML",
            SD_TAB_NEXT_BTN: "innerHTML",
        },
        vals_map={
            SD_REORDER_UP_BTN: {"direction": "up"},
            SD_REORDER_DOWN_BTN: {"direction": "down"},
            SD_TAB_PREV_BTN: {"direction": "prev"},
            SD_TAB_NEXT_BTN: {"direction": "next"},
        },
        show_hints=False,  # We render hints separately in the header
        include_state_inputs=True,  # Include keyboard navigation state in hidden inputs
    )
    
    # Build include selector for the hidden inputs
    include_selector = f"#{SD_FOCUSED_RECORD_ID_INPUT}, #{SD_FOCUSED_PROVIDER_ID_INPUT}"
    
    # Preview button is rendered separately (uses GET, triggered by on_focus_change callback)
    preview_button = Button(
        id=SD_PREVIEW_BTN,
        type="button",
        cls=str(display_tw.hidden),
        hx_get=urls.preview,
        hx_include=include_selector,
        hx_target=SelectionHtmlIds.as_selector(SelectionHtmlIds.PREVIEW_PANEL),
        hx_swap="outerHTML",
    )
    
    # Build only the active tab's content
    if active_tab == "db":
        active_content = _render_source_browser(
            transcriptions=transcriptions,
            sources=sources,
            selected_sources=selected_sources,
            add_url=urls.add,
            remove_url=urls.remove,
            preview_url=urls.preview,
            select_all_url=urls.select_all,
            filter_url=urls.filter,
            grouping_mode=grouping_mode,
            grouping_change_url=urls.grouping_change,
        )
    else:
        # Render local files browser with file browser library
        provider = _get_step_renderer_provider()
        config = _get_step_renderer_config()
        
        # Get browser state from the serialized state dict
        browser_state = BrowserState.from_dict(file_browser_state) if file_browser_state else BrowserState(
            current_path=provider.get_home_path()
        )
        
        active_content = _render_local_files_browser(
            browser_state=browser_state,
            external_paths=external_db_paths,
            provider=provider,
            config=config,
            navigate_url=urls.browse_directory,
            select_url=urls.add_external,
            remove_url=urls.remove_external,
            home_path=provider.get_home_path(),
        )
    
    # Build tabs with single shared content container
    source_tabs = _render_source_tabs(
        active_tab=active_tab,
        active_content=active_content,
        tab_switch_url=urls.tab_switch,
    )
    
    # Build queue panel
    queue_panel = _render_selection_queue(
        selected_sources=selected_sources,
        remove_url=urls.remove,
        reorder_url=urls.reorder,
        clear_url=urls.clear,
    )
    
    # Build collapsible preview panel (initially closed with no content)
    preview_panel = _render_preview_panel()
    
    # Build footer
    footer = _render_selection_footer(selected_sources, transcriptions)
    
    # Sortable.js initialization script
    sortable_script = Script(_generate_sortable_init_script())
    
    # JavaScript callbacks for keyboard navigation
    kb_callbacks_script = Script("""
        // Trigger preview update when focus changes
        function triggerPreviewUpdate(item, index, zoneId) {
            const previewBtn = document.getElementById('""" + SD_PREVIEW_BTN + """');
            if (previewBtn && item) {
                previewBtn.click();
            }
        }
        
        // Optional: log zone changes for debugging
        function onZoneChange(newZoneId, oldZoneId) {
            console.log('Zone changed from', oldZoneId, 'to', newZoneId);
        }
    """)
    
    return Div(
        # Header with keyboard hints
        Div(
            H2("Select Source Material", cls=combine_classes(font_size._3xl, font_weight.bold)),
            P(
                "Choose transcription sources to decompose into a structural spine.",
                cls=combine_classes(text_dui.base_content.opacity(70), m.b(4))
            ),
            _render_selection_keyboard_hints(kb_manager),
            cls=combine_classes(m.b(4))
        ),
        
        # Main content area (two columns: tabs + queue)
        Div(
            # Left: Source tabs with content (grows to fill space)
            source_tabs,
            
            # Right: Selection queue (fixed width)
            queue_panel,
            
            cls=combine_classes(
                grow(),
                min_h(0),  # Allow flex children to shrink below content size
                flex_display,
                flex_direction.col,  # Stack vertically on mobile
                flex_direction.row.lg,  # Horizontal on large screens
                gap(4),
                overflow.hidden,
                p(1)  # Padding to prevent focus ring clipping
            )
        ),
        
        # Collapsible preview panel (below the two columns)
        preview_panel,
        
        # Footer
        footer,
        
        # Keyboard navigation script
        kb_system.script,
        
        # Hidden inputs for keyboard navigation (library deduplicates shared inputs)
        kb_system.hidden_inputs,
        
        # Hidden HTMX buttons for keyboard actions
        kb_system.action_buttons,
        
        # Preview button (rendered separately - uses GET)
        preview_button,
        
        # JavaScript callbacks for keyboard navigation
        kb_callbacks_script,
        
        # SortableJS library
        Script(src="https://cdn.jsdelivr.net/npm/sortablejs@1.15.0/Sortable.min.js"),
        
        # Sortable initialization script (htmx integration)
        sortable_script,
        
        # Script runner for OOB-triggered JS (duplicate rejection flash)
        Div(id=SelectionHtmlIds.SCRIPT_RUNNER),
        
        id=SelectionHtmlIds.SOURCE_SELECTOR,
        cls=combine_classes(
            w.full,
            h.full,
            flex_display,
            flex_direction.col,
            p(4)
        )
    )

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()